In [ ]:
import cv2
import numpy as np
# from picarx import Picarx
from time import sleep

# px = Picarx()
px_power = 10
offset = 20
last_state = "stop"

def filter_red_color(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)
    
    return red_mask

def detect_lane_direction(image):
    height, width = image.shape
    bottom_crop = int(height * 0.95)
    bottom_section = image[bottom_crop:, :]
    white_pixels = np.where(bottom_section == 255)

    if white_pixels[0].size == 0:
        return "stop"

    left_edge_x = np.min(white_pixels[1])
    right_edge_x = np.max(white_pixels[1])
    image_center_x = width // 2

    if left_edge_x <= image_center_x <= right_edge_x:
        direction = "forward"
    elif image_center_x < left_edge_x:
        direction = "left"
    else:
        direction = "right"
    # Visualization
    image_color = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # Convert to color for drawing

    # Draw center line of the image (green)
    cv2.line(image_color, (image_center_x, 0), (image_center_x, height), (0, 255, 0), 2)

    # Draw left and right lane edges (red)
    cv2.line(image_color, (left_edge_x, bottom_crop), (left_edge_x, height), (0, 0, 255), 2)
    cv2.line(image_color, (right_edge_x, bottom_crop), (right_edge_x, height), (0, 0, 255), 2)

    # Save the visualization image
    cv2.imwrite("output.jpg", image_color)
    return direction


def outHandle():
    global last_state
    if last_state == 'left':
        px.set_dir_servo_angle(-30)
        px.backward(10)
    elif last_state == 'right':
        px.set_dir_servo_angle(30)
        px.backward(10)
    while True:
        frame = px.get_image()
        red_mask = filter_red_color(frame)
        currentSta = detect_lane_direction(red_mask)
        if currentSta != last_state:
            break
    sleep(0.001)

if __name__=='__main__':
    try:
        for i in [1]:
            ##frame = px.get_image()
            frame = cv2.imread("pics/Screenshot 2025-02-12 155221.png")
            red_mask = filter_red_color(frame)
            lane_state = detect_lane_direction(red_mask)
            print("Lane state:", lane_state)

            if lane_state != "stop":
                last_state = lane_state

            if lane_state == 'forward':
                # px.set_dir_servo_angle(0)
                # px.forward(px_power)
                print("f")
            elif lane_state == 'left':
                # px.set_dir_servo_angle(offset)
                # px.forward(px_power)
                print("l")
            elif lane_state == 'right':
                # px.set_dir_servo_angle(-offset)
                # px.forward(px_power)
                print("r")
            else:
                # outHandle()
                pass
    finally:
        # px.stop()
        print("stop and exit")
        sleep(0.1)


Lane state: left
l
stop and exit


In [ ]:
import cv2
import numpy as np
from picarx import Picarx
from time import sleep
from vilib import Vilib
import os


def filter_red_color(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)
    
    return red_mask

def detect_lane_direction(image):
    height, width = image.shape
    bottom_crop = int(height * 0.95)
    bottom_section = image[bottom_crop:, :]
    white_pixels = np.where(bottom_section == 255)

    if white_pixels[0].size == 0:
        return "stop"

    left_edge_x = np.min(white_pixels[1])
    right_edge_x = np.max(white_pixels[1])
    image_center_x = width // 2

    if left_edge_x <= image_center_x <= right_edge_x:
        direction = "forward"
    elif image_center_x < left_edge_x:
        direction = "left"
    else:
        direction = "right"

    return direction


def outHandle():
    global last_state
    if last_state == 'left':
        px.set_dir_servo_angle(-30)
        px.backward(10)
    elif last_state == 'right':
        px.set_dir_servo_angle(30)
        px.backward(10)
    while True:
        frame = px.get_image()
        red_mask = filter_red_color(frame)
        currentSta = detect_lane_direction(red_mask)
        if currentSta != last_state:
            break
    sleep(0.001)

def get_frame():
    name = 'temp'
    path = "/home/Cait/micro/Pictures"
    Vilib.take_photo(name, path)
    path = '%s/%s.jpg'%(path,name)
    image = cv2.imread(path)
    os.remove(path)
    return image

if __name__=='__main__':
    try:
        px = Picarx()
        px_power = 10
        offset = 20
        last_state = "stop"
        Vilib.camera_start(vflip=False, hflip=False)
        px.set_cam_tilt_angle(-60)
        px.set_cam_pan_angle(0)
        while True:
            frame = get_frame()
            red_mask = filter_red_color(frame)
            lane_state = detect_lane_direction(red_mask)
            print("Lane state:", lane_state)

            if lane_state != "stop":
                last_state = lane_state

            if lane_state == 'forward':
                px.set_dir_servo_angle(0)
                px.forward(px_power)
            elif lane_state == 'left':
                px.set_dir_servo_angle(offset)
                px.forward(px_power)
            elif lane_state == 'right':
                px.set_dir_servo_angle(-offset)
                px.forward(px_power)
            else:
                outHandle()
    finally:
        px.stop()
        print("stop and exit")
        sleep(0.1)
